In [1]:
import os
import re
import json
import utils
import scipy
import torch
import random
import gensim
import warnings

import numpy as np
import pandas as pd

from tasks import *
from pprint import pprint
from transformers import *
from tqdm.notebook import tqdm
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors

In [2]:
complete_df = pd.read_csv("data/clean_df.csv")
complete_df.shape

(40152, 11)

In [3]:
complete_df.head(2)

,Unnamed: 0,Unnamed: 0.1,paper_id,title,authors,affiliations,abstract,text,bibliography,raw_authors,raw_bibliography
0,0,0,6f005a0677f24f697c1ec3ce065ec4144519e357,Antiviral Mechanisms of Human Defensins Introd...,"Sarah S Wilson, Mayim E Wiens, Jason G Smith","Sarah S Wilson (University of Washington, 1705...",Defensins are an effector component of the inn...,Defensins are one of the most abundant classes...,Direct inactivation of viruses by human granul...,"[{'first': 'Sarah', 'middle': ['S'], 'last': '...","{'BIBREF0': {'ref_id': 'b0', 'title': 'Direct ..."
1,1,1,4994fa72322bbf19120592304d92629226948d8e,Rapid Identification of Malaria Vaccine Candid...,"V Villard, G W Agak, G Frank, A Jafarshad, C S...","V Villard, G W Agak, G Frank, A Jafarshad, C S...",To identify malaria antigens for vaccine devel...,Human Plasmodium falciparum (Pf) infection is ...,Identification of vaccine candidates against s...,"[{'first': 'V', 'middle': [], 'last': 'Villard...","{'BIBREF0': {'ref_id': 'b0', 'title': 'Identif..."


In [4]:
complete_df.describe()

,Unnamed: 0,Unnamed: 0.1
count,40152.000000,40152.000000
mean,29705.053248,29705.053248
std,17183.494309,17183.494309
min,0.000000,0.000000
25%,14787.750000,14787.750000
50%,29717.000000,29717.000000
75%,44551.250000,44551.250000
max,59560.000000,59560.000000


In [5]:
# Keep only texts with minimal number of words 

complete_df = complete_df[complete_df['text'].apply(lambda x: len(re.findall(r"(?i)\b[a-z]+\b", x))) > 1000]                                         
complete_df.shape

(38265, 11)

In [6]:
frac_of_articles = 1
train_df  = complete_df.sample(frac = frac_of_articles, random_state = 42)
train_corpus = (list(utils.read_corpus(train_df, 'abstract'))) 

In [7]:
# Using distributed memory model

model = gensim.models.doc2vec.Doc2Vec(dm = 1, vector_size = 50, min_count = 10, dm_mean = 1, epochs = 20, seed = 42, workers = 6)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples = model.corpus_count, epochs = model.epochs)

In [8]:
list_of_tasks = [task_1, task_2, task_3, task_4, task_5, task_6, task_7, task_8, task_9]

In [9]:
abstract_vectors = model.docvecs.vectors_docs
array_of_tasks = [utils.get_doc_vector(task, model) for task in list_of_tasks]

In [10]:
train_df['abstract_vector'] = [vec for vec in abstract_vectors]

### Nearest Neigbors search

In [11]:
train_df = train_df[train_df['abstract'].apply(lambda x: len(re.findall(r"(?i)\b[a-z]+\b", x))) > 40]
train_df.shape

(37302, 12)

In [12]:
train_array = train_df['abstract_vector'].values.tolist()

In [13]:
ball_tree = NearestNeighbors(algorithm = 'ball_tree', leaf_size = 20).fit(train_array)

In [14]:
# Query for all tasks

distances, indices = ball_tree.kneighbors(array_of_tasks, n_neighbors = 3)

In [15]:
print("="*80, f"\n\nTask = {list_of_tasks[3]}\n", )
df =  train_df.iloc[indices[3]]
abstracts = df['abstract']
titles = df['title']
dist = distances[3]
for l in range(len(dist)):
    print(f" Text index = {indices[3][l]} \n Distance = {distances[3][l]} \n Title: {titles.iloc[l]} \n Abstract extract: {abstracts.iloc[l]}\n\n")


Task = What do we know about vaccines and therapeutics? What has been published concerning research and development and evaluation efforts of vaccines and therapeutics?
Effectiveness of drugs being developed and tried to treat COVID-19 patients.
Clinical and bench trials to investigate less common viral inhibitors against COVID-19 such as naproxen, clarithromycin, and minocyclinethat that may exert effects on viral replication.
Methods evaluating potential complication of Antibody-Dependent Enhancement (ADE) in vaccine recipients.
Exploration of use of best animal models and their predictive value for a human vaccine.
Capabilities to discover a therapeutic (not vaccine) for the disease, and clinical effectiveness studies to discover therapeutics, to include antiviral agents.
Alternative models to aid decision makers in determining how to prioritize and distribute scarce, newly proven therapeutics as production ramps up. This could include identifying approaches for expanding productio

## Level 2 Abstraction using SciBERT

In [16]:
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
transformer = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased').to('cuda')

In [33]:
number_top_matches = 3

In [54]:
def convert(sentence):
    tokenized_sentence = tokenizer.tokenize(sentence)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_sentence)
    with torch.no_grad():
        vector = transformer(torch.LongTensor([indexed_tokens]).to('cuda'))[1].cpu().numpy().tolist()
    return vector[0]

In [55]:
query = 'What are the possible medications against COVID-19?' 
query_embeddings = convert(query)

In [62]:
print("="*80, f"\n\nTask = \n\n {list_of_tasks[3]}\n", )

print("\n\n======================\n\n")
print("Searching in Abstracts")
    
dfs =  train_df.iloc[indices[3]]
abstracts = [i.split(".") for i in df['abstract']]
for abstract in abstracts:
    print("\n\n======================\n\n")
    print("Abstract:", '.'.join(abstract))
    abstracts_vector = [convert(i) for i in abstract if not(len(i) < 5)]
    distance = scipy.spatial.distance.cdist([query_embeddings], abstracts_vector, "cosine")[0]

    results = zip(range(len(distance)), distance)
    results = sorted(results, key = lambda x: x[1])
    
    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 3 most similar sentences are:")

    for idx, dist in results[0:number_top_matches]:
        print(abstract[idx].strip(), "(Cosine Score: %.4f)" % (1-dist))

        
print("\n\n======================\n\n")
print("Searching in Texts")

texts = [i.split(".") for i in df['text']]
for text in texts:
    print("\n\n======================\n\n")
    print("Text:", text[:100])
    text_vector = [convert(i) for i in text if not(len(i) < 5)]
    distance = scipy.spatial.distance.cdist([query_embeddings], text_vector, "cosine")[0]

    results = zip(range(len(distance)), distance)
    results = sorted(results, key = lambda x: x[1])

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 3 most similar sentences are:")

    for idx, dist in results[0:number_top_matches]:
        print(text[idx].strip(), "\n(Cosine Score: %.4f)" % (1-dist))


Task = 

 What do we know about vaccines and therapeutics? What has been published concerning research and development and evaluation efforts of vaccines and therapeutics?
Effectiveness of drugs being developed and tried to treat COVID-19 patients.
Clinical and bench trials to investigate less common viral inhibitors against COVID-19 such as naproxen, clarithromycin, and minocyclinethat that may exert effects on viral replication.
Methods evaluating potential complication of Antibody-Dependent Enhancement (ADE) in vaccine recipients.
Exploration of use of best animal models and their predictive value for a human vaccine.
Capabilities to discover a therapeutic (not vaccine) for the disease, and clinical effectiveness studies to discover therapeutics, to include antiviral agents.
Alternative models to aid decision makers in determining how to prioritize and distribute scarce, newly proven therapeutics as production ramps up. This could include identifying approaches for expanding produc





Query: What are the possible medications against COVID-19?

Top 3 most similar sentences are:
On the same day, animal lungs showed mild to moderate interstitial pneumonia and lung consolidation, but the morphological structure of the lung was roughly normal (Panel 2M and 2N) 
(Cosine Score: 0.8456)
Yo and colleagues found that infusion of convalescent plasma resulted in beneficial clinical outcomes in SARS patients [12] 
(Cosine Score: 0.8407)
, and peaked (about 1 × 10 8 TCID 50 /g lung tissue) on day 3 p 
(Cosine Score: 0.8330)




Text: ['The family Filoviridae includes two accepted genera, Ebolavirus and Marburgvirus', ' The genus Ebolavirus includes five species (each represented by a single virus): Zaire ebolavirus (Ebola virus, EBOV), Sudan ebolavirus (Sudan virus, SUDV), Reston ebolavirus (Reston virus, RESTV), Taï Forest ebolavirus (Tai Forest virus, TAFV), and Bundibugyo ebolavirus (Bundibugyo virus, BDBV)', ' The genus Marburgvirus includes a single species, Marburg marb





Query: What are the possible medications against COVID-19?

Top 3 most similar sentences are:
reformatted the genetic insert for the vector which included the addition of a cytomegalovirus (CMV)-chicken β-actin hybrid promoter, optimized codons and a consensus Kozak sequence [57] 
(Cosine Score: 0.8636)
Richardson et al 
(Cosine Score: 0.8613)
This vaccine was also up to 100% effective following a lethal EBOV infection in mice (vaccine dose-dependent) [92, 93] 
(Cosine Score: 0.8469)




Text: ['EVD outbreaks, of which the first on 8 May (May-June 2018Equateur province DRC outbreak) and the second on 1 August (August-present, 2018, Kivu province DRC outbreak)', ' The first outbreak resulted in 54 EVD cases (38 confirmed and 16 probable) and 33 deaths (overall case-fatality ratio of 61%) as of 24 July, 2018 (declared the end of this outbreak), 29 while the latter is ongoing, which includes 238 cases (203 confirmed and 35 probable) and 155 deaths resulting in a case-fatality rate of 





Query: What are the possible medications against COVID-19?

Top 3 most similar sentences are:
These approaches help to predict the tertiary structure of a protein through its amino acid sequence to combat this issue 
(Cosine Score: 0.8475)
Critically targeting the viral lifecycle is among the most popular strategies for EBOV therapeutics 
(Cosine Score: 0.8374)
Several studies are reporting the use of quantitative structure-activity relationship modeling in lead optimization 
(Cosine Score: 0.8329)
